In [1]:
'''
Data pre process

@author:
Chong Chen (cstchenc@163.com)

@ created:
25/8/2017
@references:
'''
import os
import json
import pandas as pd
import dill as pickle
import numpy as np
TPS_DIR = '../data'
TP_file = os.path.join(TPS_DIR, 'Musical_Instruments_5.json')
#TP_file = os.path.join(TPS_DIR, 'Amazon_Instant_Video_5.json')
#TP_file = os.path.join(TPS_DIR, 'Automotive_5.json')

f= open(TP_file)
users_id=[]
items_id=[]
ratings=[]
reviews=[]
np.random.seed(2017)

for line in f:
    js=json.loads(line)
    if str(js['reviewerID'])=='unknown':
        print ("unknown")
        continue
    if str(js['asin'])=='unknown':
        print ("unknown2")
        continue
    reviews.append(js['reviewText'])
    users_id.append(str(js['reviewerID'])+',')
    items_id.append(str(js['asin'])+',')
    ratings.append(str(js['overall']))
data=pd.DataFrame({'user_id':pd.Series(users_id),
                   'item_id':pd.Series(items_id),
                   'ratings':pd.Series(ratings),
                   'reviews':pd.Series(reviews)})[['user_id','item_id','ratings','reviews']]

data.head()

,user_id,item_id,ratings,reviews
0,"A2IBPI20UZIR0U,","1384719342,",5.0,"Not much to write about here, but it does exac..."
1,"A14VAT5EAX3D9S,","1384719342,",5.0,The product does exactly as it should and is q...
2,"A195EZSQDW3E21,","1384719342,",5.0,The primary job of this device is to block the...
3,"A2C00NNG1ZQQG2,","1384719342,",5.0,Nice windscreen protects my MXL mic and preven...
4,"A94QU4C90B1AX,","1384719342,",5.0,This pop filter is great. It looks and perform...


In [2]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id, 'ratings']].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count
    
usercount, itemcount = get_count(data, 'user_id'), get_count(data, 'item_id')

print(usercount[:5])
print(itemcount[:5])

user_id
A00625243BI8W1SSZNLMD,    8
A10044ECXDUVKS,           6
A102MU6ZC9H1N6,           6
A109JTUZXO61UY,           5
A109ME7C09HM2M,           5
dtype: int64
item_id
1384719342,    5
B00004Y2UT,    6
B00005ML71,    5
B000068NSX,    7
B000068NTU,    5
dtype: int64


In [3]:
def numerize(tp):
    usercount, itemcount = get_count(data, 'user_id'), get_count(data, 'item_id')
    
    unique_uid = usercount.index
    unique_sid = itemcount.index
    item2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
    user2id = dict((uid, i) for (i, uid) in enumerate(unique_uid))

    uid = []
    sid = []
    for user in tp.user_id:
        uid.append(user2id[user])
        
    for item in tp.item_id:
        sid.append(item2id[item])
    
    tp['user_id'] = uid
    tp['item_id'] = sid
    return tp

In [4]:
data=numerize(data)
tp_rating=data[['user_id','item_id','ratings']]
tp_rating.head()

,user_id,item_id,ratings
0,550,0,5.0
1,55,0,5.0
2,92,0,5.0
3,482,0,5.0
4,1145,0,5.0


In [5]:
n_ratings = tp_rating.shape[0]
test = np.random.choice(n_ratings, size=int(0.20 * n_ratings), replace=False)
test_idx = np.zeros(n_ratings, dtype=bool)
test_idx[test] = True

In [6]:
tp_1 = tp_rating[test_idx]
tp_train= tp_rating[~test_idx]

data2=data[test_idx]
data=data[~test_idx]

n_ratings = tp_1.shape[0]
test = np.random.choice(n_ratings, size=int(0.50 * n_ratings), replace=False)

test_idx = np.zeros(n_ratings, dtype=bool)
test_idx[test] = True

tp_test = tp_1[test_idx]
tp_valid = tp_1[~test_idx]
tp_train.to_csv(os.path.join(TPS_DIR, 'music_train.csv'), index=False,header=None)
tp_valid.to_csv(os.path.join(TPS_DIR, 'music_valid.csv'), index=False,header=None)
tp_test.to_csv(os.path.join(TPS_DIR, 'music_test.csv'), index=False,header=None)

user_reviews={}
item_reviews={}
user_rid={}
item_rid={}
for i in data.values:
    if i[0] in user_reviews:
    #if user_reviews.has_key(i[0]):
        user_reviews[i[0]].append(i[3])
        user_rid[i[0]].append(i[1])
    else:
        user_rid[i[0]]=[i[1]]
        user_reviews[i[0]]=[i[3]]
    if i[1] in item_reviews:
    #if item_reviews.has_key(i[1]):
        item_reviews[i[1]].append(i[3])
        item_rid[i[1]].append(i[0])
    else:
        item_reviews[i[1]] = [i[3]]
        item_rid[i[1]]=[i[0]]


for i in data2.values:
    if i[0] in user_reviews:
    #if user_reviews.has_key(i[0]):
        l=1
    else:
        user_rid[i[0]]=[0]
        user_reviews[i[0]]=['0']
    if i[1] in item_reviews:
    #if item_reviews.has_key(i[1]):
        l=1
    else:
        item_reviews[i[1]] = [0]
        item_rid[i[1]]=['0']

pickle.dump(user_reviews, open(os.path.join(TPS_DIR, 'user_review'), 'wb'))
pickle.dump(item_reviews, open(os.path.join(TPS_DIR, 'item_review'), 'wb'))
pickle.dump(user_rid, open(os.path.join(TPS_DIR, 'user_rid'), 'wb'))
pickle.dump(item_rid, open(os.path.join(TPS_DIR, 'item_rid'), 'wb'))

usercount, itemcount = get_count(data, 'user_id'), get_count(data, 'item_id')

#print (np.sort(np.array(usercount.values)))

#print (np.sort(np.array(itemcount.values)))